# 使用 simple fusion 检索器

使用 obsidian 作为数据源。

使用 Simple Fusion retriver，基于

- embedding 检索
- keyword default 检索

评估效果。

初步结论：

- 报错，无法完成，问题未知
- 个人使用可以不用融合方式
- 融合方式的基本思路
    - 多索引
    - 生成多查询语句
    - 分别做检索
    - 检索形成 top k
    - 查询



## 全局设置

In [3]:
%%time

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=Ollama(
    base_url="http://ape:11434",
    model="qwen2",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 2.1 s, sys: 276 ms, total: 2.37 s
Wall time: 1.99 s


In [6]:
%%time

# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

CPU times: user 1.49 ms, sys: 144 µs, total: 1.64 ms
Wall time: 1.45 ms


## 加载文档

In [4]:
%%time

from llama_index.readers.obsidian import ObsidianReader


documents = ObsidianReader(
    "/models/obsidian/kb/知识库"
).load_data() 

len(documents)

CPU times: user 160 ms, sys: 7.48 ms, total: 167 ms
Wall time: 170 ms


21

## 创建索引

In [5]:
%%time

from llama_index.core import KeywordTableIndex
from llama_index.core import VectorStoreIndex

keyword_index = KeywordTableIndex.from_documents(
    documents
)

vector_index = VectorStoreIndex.from_documents(
    documents
)

CPU times: user 1.28 s, sys: 35.4 ms, total: 1.31 s
Wall time: 22.1 s


## fusion retriver

In [7]:
%%time

from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [keyword_index.as_retriever(), vector_index.as_retriever()],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

CPU times: user 3.98 ms, sys: 0 ns, total: 3.98 ms
Wall time: 3.61 ms


In [8]:
%%time

nodes_with_scores = retriever.retrieve("docker容器在后台执行的命令")

Generated queries:
1. 如何在Docker容器中运行后台命令
2. Docker容器后台执行命令示例及说明
3. Docker容器内后台执行命令的最佳实践和注意事项


TypeError: '>' not supported between instances of 'NoneType' and 'float'

## 查询

In [10]:
%%time

from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever,
    streaming=True
)

CPU times: user 113 µs, sys: 10 µs, total: 123 µs
Wall time: 126 µs


In [11]:
%%time

streaming_response = query_engine.query("docker容器在后台执行的命令")
streaming_response.print_response_stream()

Generated queries:
1. 如何在Docker容器中运行后台命令
2. Docker容器后台执行命令详解
3. 在Docker容器内执行后台任务的方法


TypeError: '>' not supported between instances of 'NoneType' and 'float'